In [4]:
try:
    import tweepy
    from firebase_admin import credentials, firestore, initialize_app
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
except ImportError:
    raise ImportError("La aplicación requiere las librerías tweepy, firebase_admin y transformers.")    

import datetime

In [ ]:
# Firestore
cred = credentials.Certificate('service-acc-key.json')
initialize_app(cred)
db = firestore.client()

In [24]:
incomingDict = {'total': (123, 14, 5), 'madrid': (14, 3, 2), 'barcelona': (5, 2, 1), 'valencia': (1, 1, 1)}

In [26]:
for city, results in incomingDict.items():
    db.collection('test').document('testUserID').collection('2022-05-28').document(city).set({
        'pos': results[0],
        'neut': results[1],
        'neg': results[2]
    })

In [18]:
ref = db.collection('test').document('testUserID').collection('2022-05-28')

In [19]:
ref2 = ref.document('test1')

In [20]:
doc = ref2.get()

In [21]:
doc.exists

False

In [22]:
# firestore create document
ref2.set({'pos': 25, 'neg': 14, 'neut': 5})

update_time {
  seconds: 1654363323
  nanos: 725073000
}

In [8]:
ref.update({'isPotato': False})

NotFound: 404 No document to update: projects/firebirdbackend/databases/(default)/documents/test/apple

In [ ]:
campaigns = db.collection('campaigns').stream()

for doc in campaigns:

    campaign = doc.to_dict()

    # Ejemplo de formato:
    # Campaign atleti: 
    # {'track': '#atleti', 
    # 'isActive': True, 
    # 'tweet_limit': 1000, 
    # 'duration': 100, 
    # 'start': DatetimeWithNanoseconds(2022, 4, 19, 22, 0, tzinfo=datetime.timezone.utc),
    # 'userID': '.......'}

    

    if campaign['isActive']:

        # Comprobar que la campaña sigue estando activa
        # En caso contrario, marcarla como inactiva y pasar a la siguiente

        if campaign['start'] + datetime.timedelta(days = campaign['duration']) > datetime.datetime.now():
            doc.update({'isActive': False})
            continue

        query = campaign['track']
        numTweets = campaign['tweet_limit']
        userID = campaign['userID']

        doc_ref = db.collection('public').document('userID')
        doc = doc_ref.get()
        if doc.exists:
            logging.info("Found")
        else:
            logging.info("Not found")